In [ ]:
// 13. Write CUDA program for matrix multiplication

In [1]:
pip install nvcc4jupyter

In [2]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpinhmyqo0".


In [3]:
%%cuda
#include <iostream>

#define N 3

__global__ void matrixMul(int *a, int *b, int *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        int sum = 0;
        for (int i = 0; i < N; ++i) {
            sum += a[row * N + i] * b[i * N + col];
        }
        c[row * N + col] = sum;
    }
}

int main() {
    int a[N][N] = { {1, 2, 3}, {4, 5, 6}, {7, 8, 9} };
    int b[N][N] = { {9, 8, 7}, {6, 5, 4}, {3, 2, 1} };
    int c[N][N] = {0};

    int *dev_a, *dev_b, *dev_c;
    cudaMalloc((void**)&dev_a, N * N * sizeof(int));
    cudaMalloc((void**)&dev_b, N * N * sizeof(int));
    cudaMalloc((void**)&dev_c, N * N * sizeof(int));

    cudaMemcpy(dev_a, a, N * N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, N * N * sizeof(int), cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(N, N);
    dim3 numBlocks(1, 1);

    matrixMul<<<numBlocks, threadsPerBlock>>>(dev_a, dev_b, dev_c);

    cudaMemcpy(c, dev_c, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(dev_a); cudaFree(dev_b); cudaFree(dev_c);

    std::cout << "Resultant Matrix:" << std::endl;
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            std::cout << c[i][j] << " ";
        }
        std::cout << std::endl;
    }

    return 0;
}


Resultant Matrix:
30 24 18 
84 69 54 
138 114 90 



To run

Google Colab--> Runtime--> change runtime type--> Hardware accelerator--> T4 GPU--> save--> (in new cell) pip install nvcc4jupyter--> run(Shift+enter)--> (In new cell) %load_ext nvcc4jupyter--> run(Shift+enter)--> (in new cell) write program(Start with %%cuda)--> code--> shift+enter